In [73]:
#libraries
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend
from keras.utils import np_utils
from keras import regularizers
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.engine.input_layer import Input
from keras.models import Model
from keras.optimizers import SGD, Adam

#tensorflow2 libs
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate, ZeroPadding2D, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam

In [74]:
# dimensions of our images.
img_width, img_height = 64, 64

In [75]:
#declarations of dataset specs
train_data_dir = 'data/train'
test_data_dir = 'data/test'
nb_train_samples = 4672
nb_validation_samples = 660
epochs = 64
batch_size = 8
num_classes = 7

In [76]:
#adding zero padding in images
top_model = Sequential()                                                                                                                                                                              
top_model.add(ZeroPadding2D((80, 80), input_shape=(64, 64, 3)))

In [ ]:
#Resnet50
new_input = Input(shape=(224, 224, 3))

base_model = tf.keras.applications.ResNet50(input_tensor=new_input, weights='imagenet', include_top=False)

#freezing layers for regularization
#for i in range(len(base_model.layers)-1):
    #base_model.layers[i].trainable = False


# add new classifier layers
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.7)(x)

# predictions = Dense(num_classes, activation= 'softmax')(x)
# ResNet50_mod = Model(inputs = base_model.input, outputs = predictions)

# define new model
# model.add(ResNet50_mod)

# model.summary()

In [77]:
#top model to be trained 

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[7:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(7, activation='sigmoid'))



In [78]:
#compile model
top_model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])



In [79]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [80]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1. / 255)

In [81]:
#inputting the dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 4672 images belonging to 7 classes.
Found 660 images belonging to 7 classes.


In [82]:
#running the model
history = top_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


  ...
    to  
  ['...']
Train for 584 steps, validate for 82 steps
Epoch 1/64
  1/584 [..............................] - ETA: 18s

ValueError: logits and labels must have the same shape ((None, None, None, 7) vs (None, None))

In [ ]:
#putting data into variables
x_train, y_train = train_generator.next()
x_test, y_test = validation_generator.next()

In [ ]:

#getting evaluation scores
score = model.evaluate(x_test, y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#saving weights
model.save_weights('fc_model.h5')

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()